In [2]:
from flair.data import Sentence
from flair.models import SequenceTagger

# load tagger
tagger = SequenceTagger.load("flair/ner-french")

2025-09-23 08:41:02,962 SequenceTagger predicts: Dictionary with 19 tags: O, S-LOC, B-LOC, E-LOC, I-LOC, S-PER, B-PER, E-PER, I-PER, S-MISC, B-MISC, E-MISC, I-MISC, S-ORG, B-ORG, E-ORG, I-ORG, <START>, <STOP>
Sentence[24]: "M. et Mme. JOUBERT Alain et Vivianne déclarent être propriétaire du logement enquêté et que ce dernier n'est pas vacant car loué." → ["Mme"/PER, "JOUBERT Alain"/PER, "Vivianne"/PER]
The following NER tags are found:
Span[2:3]: "Mme" → PER (0.6625)
Span[4:6]: "JOUBERT Alain" → PER (0.6730)
Span[7:8]: "Vivianne" → PER (0.9510)


In [ ]:
# make example sentence
sentence = Sentence("M. et Mme. JOUBERT Alain et Vivianne déclarent être propriétaire du logement enquêté et que ce dernier n'est pas vacant car loué.")

# predict NER tags
tagger.predict(sentence)

# print sentence
print(sentence)

# print predicted NER spans
print('The following NER tags are found:')
# iterate over entities and print
for entity in sentence.get_spans('ner'):
    print(entity)

In [3]:
path = '/Users/raphaelcourivaud/dev/zlv/export.tsv'

In [4]:
import pandas as pd

In [6]:
df = pd.read_csv(path, sep='\t')

In [18]:
sentence = Sentence(df.iloc[10]["Notes__content"])
tagger.predict(sentence)
print(sentence)
for entity in sentence.get_spans('ner'):
    print(entity)

Sentence[19]: "Mme. OLLIVIER Patricia déclare être propriétaire du logement enquêté et que celui-ci n'est pas vacant car loué." → ["Mme"/PER, "OLLIVIER Patricia"/PER]
Span[0:1]: "Mme" → PER (0.4865)
Span[2:4]: "OLLIVIER Patricia" → PER (0.8556)


In [30]:
a = sentence.get_spans()[0]


'PER'

In [14]:
import faker

In [15]:
fake = faker.Faker("fr_FR")

In [16]:
fake.name()

'Jeanne Thibault-Benoit'

In [58]:
def _clean_preprocess(x: str):
    return x.replace("M.", "").replace("Mme.", "").replace("M ", "").replace("Mme ", "").strip()

In [109]:
def _predict_and_replace(x: str):
    sentence = Sentence(x)
    tagger.predict(sentence)
    anonymized_text = x 
    for span in sentence.get_spans():
        end_at = span.end_position
        start_at = span.start_position
        tag = span.tag
        print(span)
        if tag == 'PER':
            replacement = "<NOM>" # fake.name()
        elif tag == 'LOC':
            replacement = "<VILLE>" # fake.city()
        elif tag == 'ORG':
            replacement = "<ENTREPRISE>" # fake.city()
        elif tag == 'MISC':
            replacement = anonymized_text[start_at:end_at]
        else:
            continue
        anonymized_text = anonymized_text[:start_at] +" " +  replacement +" " + anonymized_text[end_at:]
            
    return anonymized_text.strip().replace("  ", " ")

In [110]:
df.head()["Notes__content"][0]

"M. et Mme. JOUBERT Alain et Vivianne déclarent être propriétaire du logement enquêté et que ce dernier n'est pas vacant car loué."

In [111]:
content = df.iloc[896]["Notes__content"]
filtered_content = _clean_preprocess(content)
result = _predict_and_replace(filtered_content)
print(f"{content} \n\n ========= \n\n{result}")

Span[81:83]: "Sincères salutations" → MISC (0.5726)
Span[86:88]: "SCI POMAGA" → ORG (0.6323)
Span[88:90]: "Charles MORDEL" → PER (0.5308)
Courrier entrant : " Suite à votre courrier du 18 août, veuillez trouver ci-joint le formulaire dûment rempli.
Cependant, au dire de votre courrier, ce logement a été déclaré comme inoccupé depuis 2 ans au 1er janvier 2022.
Ce logement ayant été loué depuis son acquisition en 2013 jusqu’à ce jour sans discontinuité et sachant que nous avons fait toutes les déclarations relatives à ce logement, nous aimerions savoir qui vous a indiqué ces renseignements
Sincères salutations.
Pour la SCI POMAGA
Charles MORDEL"
 


Courrier entrant : " Suite à votre courrier du 18 août, veuillez trouver ci-joint le formulaire dûment rempli.
Cependant, au dire de votre courrier, ce logement a été déclaré comme inoccupé depuis 2 ans au 1er janvier 2022.
Ce logement ayant été loué depuis son acquisition en 2013 jusqu’à ce jour sans discontinuité et sachant que nous avons f